In [1]:
import keras
import pandas as pd
import tensorflow as tf
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from data_generator import DataGenerator

import segmentation_models as sm
from segmentation_models.metrics import IOUScore, FScore
from segmentation_models.losses import DiceLoss, BinaryFocalLoss


from keras.models import model_from_json

sm.set_framework('tf.keras')
sm.framework()

# setting dynamic memory growth for GPU VRAM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [3]:
# Defining Hyper-parameters

BACKBONE = 'efficientnetb4'
CLASSES = ['floor']
LR = 0.0001
EPOCHS = 10
batch_size = 2
input_shape=(288, 288, 3) # divisible by 32

preprocess_input = sm.get_preprocessing(BACKBONE)

In [4]:
# Defining network parameters
n_classes = 1
activation = 'sigmoid' 

In [5]:
# Create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation, input_shape=input_shape)

# Define optomizer
optim = tf.keras.optimizers.Adam(LR)

# Combining segmentation model losses
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# Compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

In [6]:
# Creating data generators for train and valid

train_df = pd.read_csv( 'training.csv')
train_gen = DataGenerator(train_df,
                          X_col='rgb',
                          y_col='mask',
                          batch_size=batch_size, 
                          input_size=input_shape,
                          shuffle=True)

valid_df  = pd.read_csv('validation.csv')
valid_gen  = DataGenerator(valid_df,
                          X_col='rgb',
                          y_col='mask',
                          batch_size=batch_size,
                          input_size=input_shape,
                          shuffle=False)


In [7]:
# # Loading sample from data generator

# X, y = valid_gen.get_sample(8)

# # Image
# plt.imshow(X)
# plt.show()
# # Ground truth
# plt.imshow(y)
# plt.show()

In [8]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", restore_best_weights=True, mode='min', patience=15, verbose=1),
    keras.callbacks.ModelCheckpoint('checkpoints/{val_loss:.4f}.h5', monitor="val_loss", mode="min", save_best_only=True, save_weights_only=False,  verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=0, mode='min', min_delta=0.0001, cooldown=0, min_lr=0)
]

In [9]:
# train the model
history = model.fit(
    train_gen,  
    epochs=EPOCHS,
    callbacks=callbacks, 
    validation_data=valid_gen,
    batch_size=batch_size
)

Epoch 1/10
4989/4989 [==============================] - ETA: 0s - loss: 0.3591 - iou_score: 0.6394 - f1-score: 0.7544
Epoch 1: val_loss improved from inf to 0.25240, saving model to checkpoints\0.2524.h5
4989/4989 [==============================] - 1095s 215ms/step - loss: 0.3591 - iou_score: 0.6394 - f1-score: 0.7544 - val_loss: 0.2524 - val_iou_score: 0.7170 - val_f1-score: 0.8149 - lr: 1.0000e-04
Epoch 2/10
4989/4989 [==============================] - ETA: 0s - loss: 0.2547 - iou_score: 0.7116 - f1-score: 0.8138
Epoch 2: val_loss improved from 0.25240 to 0.23483, saving model to checkpoints\0.2348.h5
4989/4989 [==============================] - 1016s 204ms/step - loss: 0.2547 - iou_score: 0.7116 - f1-score: 0.8138 - val_loss: 0.2348 - val_iou_score: 0.7331 - val_f1-score: 0.8287 - lr: 1.0000e-04
Epoch 3/10
4989/4989 [==============================] - ETA: 0s - loss: 0.2294 - iou_score: 0.7353 - f1-score: 0.8316
Epoch 3: val_loss improved from 0.23483 to 0.22851, saving model to chec

In [ ]:

# serialize model to json
json_model = model.to_json()
#save the model architecture to JSON file
with open('model_files/model.json', 'w') as json_file:
    json_file.write(json_model)